In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
from ipywidgets import widgets
from IPython.display import display, HTML
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import numpy as np


# Customized modules
HOME = '/home/serinatan/project/GPU-Virtualization-Benchmarks/util'
if HOME not in sys.path:
    sys.path.append(HOME)

import data.scripts.common.format as fmt
import data.scripts.common.constants as const
import data.scripts.common.help_iso as hi

from gpupool.workload import BatchJob
from gpupool.predict import Allocation, StageOne, StageTwo

SCRIPT_PATH = os.path.join(HOME, 'data/scripts') 
PKL_PATH = os.path.join(HOME, 'data/pickles') 
CSV_PATH = os.path.join(HOME, 'data/csv')
TIME_PATH = os.path.join(HOME, 'data/timeline')

%config InlineBackend.figure_format ='retina'
plt.style.use('seaborn-talk')

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
batch = BatchJob(200, num_benchmarks_per_job=1, num_jobs=4)

batch.calculate_gpupool_performance(Allocation.One_D, StageOne.GPUSim, StageTwo.Full, at_least_once=True)
batch.calculate_gpupool_performance(Allocation.Three_D, StageOne.GPUSim, StageTwo.Full, at_least_once=True)

batch.df_pair

[0.109375, 0.2890625, 0.46875, 0.6484375, 0.828125]


,pair_job,pair_str,One_D-GPUSim-Full-True-option,One_D-GPUSim-Full-True-perf,Three_D-GPUSim-Full-True-option,Three_D-GPUSim-Full-True-perf
0,<gpupool.predict.PairJob object at 0x7fcd2c175...,job-0+job-1,<gpupool.predict.RunOption1D object at 0x7fcd3...,<gpupool.predict.Performance object at 0x7fcd2...,<gpupool.predict.RunOption3D object at 0x7fcca...,<gpupool.predict.Performance object at 0x7fcca...
1,<gpupool.predict.PairJob object at 0x7fcd2c175...,job-0+job-2,<gpupool.predict.RunOption1D object at 0x7fccb...,<gpupool.predict.Performance object at 0x7fcca...,<gpupool.predict.RunOption3D object at 0x7fcca...,<gpupool.predict.Performance object at 0x7fcca...
2,<gpupool.predict.PairJob object at 0x7fcd2c175...,job-0+job-3,<gpupool.predict.RunOption1D object at 0x7fccb...,<gpupool.predict.Performance object at 0x7fcca...,<gpupool.predict.RunOption3D object at 0x7fcca...,<gpupool.predict.Performance object at 0x7fcca...
3,<gpupool.predict.PairJob object at 0x7fcd2c175...,job-1+job-2,<gpupool.predict.RunOption1D object at 0x7fcca...,<gpupool.predict.Performance object at 0x7fcca...,<gpupool.predict.RunOption3D object at 0x7fcd2...,<gpupool.predict.Performance object at 0x7fcca...
4,<gpupool.predict.PairJob object at 0x7fcd2c175...,job-1+job-3,<gpupool.predict.RunOption1D object at 0x7fcca...,<gpupool.predict.Performance object at 0x7fcca...,<gpupool.predict.RunOption3D object at 0x7fcca...,<gpupool.predict.Performance object at 0x7fcca...
5,<gpupool.predict.PairJob object at 0x7fcd2c175...,job-2+job-3,<gpupool.predict.RunOption1D object at 0x7fcca...,<gpupool.predict.Performance object at 0x7fcd3...,<gpupool.predict.RunOption3D object at 0x7fcca...,<gpupool.predict.Performance object at 0x7fcca...


In [11]:
row = 2
for job in batch.df_pair.iloc[row]['pair_job'].jobs:
    print(job.name, job.benchmarks)

print('\n')
print("ctx", batch.df_pair.iloc[row]['One_D-GPUSim-Full-True-option'].ctx)
print('\n')
batch.df_pair.iloc[row]['One_D-GPUSim-Full-True-option'].print_config()
batch.df_pair.iloc[row]['One_D-GPUSim-Full-True-option'].print_interference()
batch.df_pair.iloc[row]['One_D-GPUSim-Full-True-perf'].visualize("1D").display()
print('sld:', batch.df_pair.iloc[row]['One_D-GPUSim-Full-True-perf'].sld)
print('ws:', sum(batch.df_pair.iloc[row]['One_D-GPUSim-Full-True-perf'].sld))

job-0 ['rod_lavamd-0']
job-3 ['rod_heartwall-0']


ctx [0.828125, 0.171875]


job-0 Config
                 rod_lavamd-0
rod_heartwall-0             7
----------------------------------------------------------------------------------------------------
job-3 Config
                 rod_lavamd-0
rod_heartwall-0             1
----------------------------------------------------------------------------------------------------
job-0 Interference
                 rod_lavamd-0
rod_heartwall-0      1.077682
----------------------------------------------------------------------------------------------------
job-3 Interference
                 rod_lavamd-0
rod_heartwall-0      0.890287
----------------------------------------------------------------------------------------------------


alt.LayerChart(...)

sld: [1.077682324525946, 0.8902873660567373]
ws: 1.9679696905826831


In [12]:
for job in batch.df_pair.iloc[row]['pair_job'].jobs:
    print(job.name, job.benchmarks)


print('\n')
batch.df_pair.iloc[row]['Three_D-GPUSim-Full-True-option'].print_config()
batch.df_pair.iloc[row]['Three_D-GPUSim-Full-True-option'].print_interference()
batch.df_pair.iloc[row]['Three_D-GPUSim-Full-True-perf'].visualize("3D").display()
print('sld:', batch.df_pair.iloc[row]['Three_D-GPUSim-Full-True-perf'].sld)
print('ws:', sum(batch.df_pair.iloc[row]['Three_D-GPUSim-Full-True-perf'].sld))

job-0 ['rod_lavamd-0']
job-3 ['rod_heartwall-0']


job-0 Config
                 rod_lavamd-0
rod_heartwall-0             6
----------------------------------------------------------------------------------------------------
job-3 Config
                 rod_lavamd-0
rod_heartwall-0             1
----------------------------------------------------------------------------------------------------
job-0 Interference
                 rod_lavamd-0
rod_heartwall-0      1.077682
----------------------------------------------------------------------------------------------------
job-3 Interference
                 rod_lavamd-0
rod_heartwall-0      0.890287
----------------------------------------------------------------------------------------------------


alt.LayerChart(...)

sld: [1.077682324525946, 0.8902873660567373]
ws: 1.9679696905826831


In [10]:
batch.calculate_gpupool_performance(Allocation.One_D, StageOne.GPUSim, StageTwo.Full, at_least_once=True)

[0.109375, 0.890625] [0.5231282105997863, 0.9386897315621358]
[0.2890625, 0.7109375] [0.5231282105997863, 0.9386897315621358]
[0.46875, 0.53125] [0.9006468690969325, 0.9188303420345404]
[0.6484375, 0.3515625] [0.9006468690969325, 0.9188303420345404]
[0.828125, 0.171875] [1.077682324525946, 0.8902873660567373]
best_perf: <gpupool.predict.Performance object at 0x7fccac08e4a8>
idx 4
ctx [0.828125, 0.171875]
